    HOT WIRE ANEMOMETER CALIBRATION v.3
    LEA - Wind Tunnel
    python version

Developer: Rafael C. F. Mendes

Date:29/06/2023   (v1: 24/11/2016)

Description: Use this program to move hot wire probe and taking data from CTA module.
             
Hardware: 
   - Arduino Uno (probe positioning system) with GRBL library (see more at https://github.com/grbl/grbl)
   - National Instrument - USB-4332 **Warning: Working voltage -10 to 10V**                 
   - Hot wire anemometer - miniCTA DANTEC

Before run it, install the following libraries:

    - pip install nidaqmx
    - pip install pyserial

In [19]:
import nidaqmx          #pip install nidaqmx
import time
import numpy as np
import serial           #pip install pyserial
from datetime import datetime

from nidaqmx.constants import LineGrouping
from nidaqmx.constants import Edge
from nidaqmx.constants import AcquisitionType 

def acquire_data(SampleRate, Sample):
    with nidaqmx.Task() as master_task:
        master = master_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
        master_task.timing.cfg_samp_clk_timing(SampleRate, source='', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)
        master_task.stop()

        master_data = master_task.read(Sample)
    
    return master_data


SampleRate=1000
AquisitionTimeInSeconds=10
Sample=SampleRate*AquisitionTimeInSeconds

data=acquire_data(SampleRate, Sample)
print(data)
"""""
with nidaqmx.Task() as master_task:
    master = master_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")
    master_task.timing.cfg_samp_clk_timing(SampleRate, source='', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)
    master_task.stop()

    master_data = master_task.read(AquisitionTimeInSeconds)
"""""


'""\nwith nidaqmx.Task() as master_task:\n    master = master_task.ai_channels.add_ai_voltage_chan("Dev1/ai0")\n    master_task.timing.cfg_samp_clk_timing(SampleRate, source=\'\', active_edge=Edge.RISING, sample_mode=AcquisitionType.CONTINUOUS)\n    master_task.stop()\n\n    master_data = master_task.read(AquisitionTimeInSeconds)\n'

In [ ]:
s = serial.Serial('COM4', baudrate=9600)  # Serial port for Arduino UNO
s.open()
time.sleep(0.2)
s.write(b'x-400 y-330')
s.close()


In [ ]:
len(master_data)

9000

In [20]:


now = datetime.now()
date_string = now.strftime('%d-%b-%Y')
time_string = now.strftime('%H-%M-%S')
filename = f"calibration/{date_string}_{time_string}.txt"





# See after the input function

In [23]:
i = 1
datap = 0
press = []
Usig = []

while datap >= 0:
    x = input('mmh2o: ')  
    datap = float(x)

    if datap >= 0:
        press.append(datap)
        data = acquire_data(SampleRate, Sample)
        Usig.append(np.mean(data))
        # Tsig(i) = np.mean(data[:, 2])  # Uncomment this line if needed

    i += 1
    if datap < 0:
        break
    
n = i - 1


In [ ]:

Ec = 1  # sqrt((Twire-Treff)./(Twire-Temp))
Usig_c = Ec * Usig
U = np.sqrt(2 * 9.81 * 998 * np.array(press) / 1000 / 1.02)

fitresult, gof = createFit_king_law(Usig_c, U)
fitresult4, gof4 = createFit4(Usig_c, U)

coef_values = fitresult.params.values()
coef_values4 = fitresult4.params.values()

with open(filename, 'wt') as fid:
    fid.write('Coefficients kings law\t')
    fid.write('U(x) = (x^2/b-a)^(1/0.25)\n')
    fid.write('a \t b\n')
    fid.write(f'{coef_values[0]:12.4f} \t {coef_values[1]:12.4f}\n')

    fid.write('\nCoefficients poly4\t')
    fid.write('U(x) = p1*x^4 + p2*x^3 + p3*x^2 + p4*x + p5\n')
    fid.write('p1\t p2\t p3\t p4\t p5\n')
    fid.write(f'{coef_values4[0]:12.4f} \t {coef_values4[1]:12.4f} \t {coef_values4[2]:12.4f} \t {coef_values4[3]:12.4f} \t {coef_values4[4]:12.4f}\n')

    fid.write('\nSignal \t Pressure \t Velocity\n')
    for i in range(n-1):
        fid.write(f'{Usig_c[i]:12.8f} \t {press[i]:12.8f} \t {U[i]:12.8f}\n')


In [ ]:
s.open()
time.sleep(0.2)
s.write(b'$H')
s.close()
